# Processing the All-atom Data

## TL;DR

In [1]:
import h5py
DATA_FILE = "/import/a12/users/kraemea88/two_peptides/data/allatom.h5"
with h5py.File(DATA_FILE, "r") as data:
    print(data["MINI"])
    print("Available Data:", list(data["MINI"]["IL_LF"].keys()))
    print("Coordinate shape:", data["MINI"]["IL_LF"]["aa_coords"].shape)
    
    from two_peptides.h5 import string_to_topology
    print(string_to_topology(data["MINI"]["IL_LF"].attrs["topology"]))
   

<HDF5 group "/MINI" (1213 members)>
Available Data: ['aa_coords', 'aa_forces', 'bias_energy', 'box_vectors', 'd0', 'distance', 'k', 'unbiased_energy']
Coordinate shape: (27000, 101, 3)
<mdtraj.Topology with 2 chains, 8 residues, 101 atoms, 124 bonds>


## Read the Data

In [2]:
DATA_FILE = "/import/a12/users/kraemea88/two_peptides/data/allatom.h5"

import h5py
ALL_ATOM_FILE = h5py.File(DATA_FILE, "r")
ALL_ATOM_DATA = ALL_ATOM_FILE["MINI"]


You can look at all the pairs in the data as `ALL_ATOM_DATA.keys()`. For each pair, there are the following data: 


In [3]:
PEPTIDE_PAIR = "IL_LF"
ALL_ATOM_DATA[PEPTIDE_PAIR].keys()

<KeysViewHDF5 ['aa_coords', 'aa_forces', 'bias_energy', 'box_vectors', 'd0', 'distance', 'k', 'unbiased_energy']>

as well as metadata (the mdtraj topology)

In [4]:
ALL_ATOM_DATA[PEPTIDE_PAIR].attrs.keys()

<KeysViewHDF5 ['topology']>

## Access the topology

In [5]:
from two_peptides.h5 import string_to_topology
TOPOLOGY = string_to_topology(ALL_ATOM_DATA[PEPTIDE_PAIR].attrs["topology"])

## Define the embedding

In [6]:
from two_peptides.meta import embedding_map
import mdtraj as md 

print("The embedding_map from the transferable CG project:", embedding_map)


def embedding(atom: md.core.topology.Atom) -> int:
    
    # cap embeddings
    #ace_embedding = {"CH3": 25, "C": 26, "O": 27}
    #nme_embedding = {"N": 28, "C": 29}
    ace_embedding = {"C": 25}
    nme_embedding = {"N": 26}
    
    if atom.residue.name == "NME":
        return nme_embedding.get(atom.name, None)
    elif atom.residue.name == "ACE":
        return ace_embedding.get(atom.name, None)
    elif atom.residue.name in embedding_map:
        # Heavy-backbone embedding from the transferable CG project
        if atom.name in ["N", "CA", "C", "O"]:
            return embedding_map[atom.name]
        elif atom.name == "CB":
            return embedding_map[atom.residue.name]
    else:
        return None
    

The embedding_map from the transferable CG project: {'ALA': 1, 'CYS': 2, 'ASP': 3, 'GLU': 4, 'PHE': 5, 'GLY': 6, 'HIS': 7, 'ILE': 8, 'LYS': 9, 'LEU': 10, 'MET': 11, 'ASN': 12, 'PRO': 13, 'GLN': 14, 'ARG': 15, 'SER': 16, 'THR': 17, 'VAL': 18, 'TRP': 19, 'TYR': 20, 'N': 21, 'CA': 22, 'C': 23, 'O': 24}


In [7]:
BEADS = [atom.index for atom in TOPOLOGY.atoms if embedding(atom) is not None]
CG_TOPOLOGY = TOPOLOGY.subset(BEADS)

In [8]:
from typing import Callable
import numpy as np

def make_coordinate_mapping(topology: md.Topology, embedding: Callable) -> np.ndarray:
    """slice mapping based on the embedding"""
    beads = [atom.index for atom in topology.atoms if embedding(atom) is not None]
    coordinate_mapping = (np.array(beads)[..., None] == np.arange(topology.n_atoms))
    return coordinate_mapping.astype(float)

COORDINATE_MAPPING = make_coordinate_mapping(TOPOLOGY, embedding)

In [9]:
def make_force_mapping(topology: md.Topology, coordinate_mapping: np.ndarray) -> np.ndarray:
    """basic aggregation of hydrogens"""
    force_mapping = coordinate_mapping.copy()
    for atom1, atom2 in topology.bonds:
        if atom1.element == md.core.element.hydrogen:
            heavy, hydrogen = atom2, atom1
        elif atom2.element == md.core.element.hydrogen:
            heavy, hydrogen = atom1, atom2
        else:
            continue
        force_mapping[:, hydrogen.index] = force_mapping[:, heavy.index]
    return force_mapping

FORCE_MAPPING = make_force_mapping(TOPOLOGY, COORDINATE_MAPPING)

## Process all-atom data

In [10]:
CG_COORDINATES = COORDINATE_MAPPING @ ALL_ATOM_DATA[PEPTIDE_PAIR]["aa_coords"]
CG_COORDINATES.shape

(27000, 24, 3)

In [11]:
FORCE_MAPPING = COORDINATE_MAPPING
CG_FORCES = FORCE_MAPPING @ ALL_ATOM_DATA[PEPTIDE_PAIR]["aa_forces"]

In [12]:
def process_pair(
    peptide_pair: str,
    embedding: Callable,
    output_file: h5py.File,
    input_file: h5py.File = ALL_ATOM_FILE,
):
    #read
    pair_data = input_file["MINI"][peptide_pair]
    topology = string_to_topology(pair_data.attrs["topology"])
    cg_embeds = np.array(
        [embedding(atom) 
         for atom in topology.atoms 
         if embedding(atom) is not None]
    )
    coordinate_mapping = make_coordinate_mapping(topology, embedding)
    force_mapping = make_force_mapping(topology, coordinate_mapping)
    cg_coordinates = coordinate_mapping @ pair_data["aa_coords"]
    cg_forces = force_mapping @ pair_data["aa_forces"]    
    
    #write 
    minigroup = outfile.require_group("MINI")
    pairgroup = minigroup.require_group(peptide_pair)
    pairgroup.attrs["cg_embeds"] = cg_embeds
    pairgroup.attrs["N_frames"] = cg_coordinates.shape[0]
    pairgroup.create_dataset(
        "cg_coords", 
        cg_coordinates.shape, 
        np.float32, 
        cg_coordinates
    )
    pairgroup.create_dataset(
        "cg_forces", 
        cg_forces.shape, 
        np.float32, 
        cg_forces
    )


In [ ]:
from tqdm.auto import tqdm

with h5py.File("data/test_heavybackbone.h5", "w") as outfile:
    progress_bar = tqdm(list(ALL_ATOM_DATA.keys())[:10])
    for peptide_pair in progress_bar:
        progress_bar.desc = peptide_pair
        process_pair(peptide_pair, embedding, outfile)

  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
with h5py.File("data/test_heavybackbone.h5", "r") as outfile:
    print(outfile)

<HDF5 file "test_heavybackbone.h5" (mode r)>
